In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
import os
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchtext.data.utils import get_tokenizer

from genomic_benchmarks.dataset_getters.pytorch_datasets import HumanEnhancersEnsembl
from genomic_benchmarks.dataset_getters.utils import coll_factory, LetterTokenizer, build_vocab, check_seq_lengths, check_config 

from cnn_model import CNN

## Config

In [6]:
config = {
    "use_padding": True,
    "run_on_gpu": True,
    "dataset": HumanEnhancersEnsembl,
    "number_of_classes": 2,
    "dataset_version": 0,
    "force_download": False,
    "epochs": 15,
    "embedding_dim": 100,
    "batch_size": 32,
#   vocabulary that is not present in the training set but is present in the test set
    "vocab_to_add": [],
}
check_config(config)

config is correct


## Choose the dataset

In [7]:
get_dataset_fn = config["dataset"]
train_dset = get_dataset_fn('train', force_download=config["force_download"], version=config["dataset_version"])

Unzipping...Done.


## Tokenizer and vocab

In [8]:
tokenizer = get_tokenizer(LetterTokenizer())
vocabulary = build_vocab(train_dset, tokenizer, use_padding=config["use_padding"])

print("vocab len:" ,vocabulary.__len__())
print(vocabulary.get_stoi())

vocab len: 8
{'N': 6, 'C': 4, 'A': 3, '<eos>': 5, 'G': 2, '<pad>': 7, 'T': 1, '<bos>': 0}


## Dataloader and batch preparation

In [9]:
# Run on GPU or CPU
device = 'cuda' if config["run_on_gpu"] and torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

max_seq_len, nn_input_len = check_seq_lengths(dataset=train_dset, config=config)

# Data Loader
if(config["use_padding"]):
    collate = coll_factory(vocabulary, tokenizer, device, pad_to_length = nn_input_len)
else:
    collate = coll_factory(vocabulary, tokenizer, device, pad_to_length = None)

train_loader = DataLoader(train_dset, batch_size=config["batch_size"], shuffle=True, collate_fn=collate)

Using cuda device
max_seq_len  573
not all sequences are of the same length


## Model

In [10]:
model = CNN(
    number_of_classes=config["number_of_classes"],
    vocab_size=vocabulary.__len__(),
    embedding_dim=config["embedding_dim"],
    input_len=nn_input_len
).to(device)

/opt/conda/envs/bench_env/lib/python3.8/site-packages/torch/nn/functional.py:652: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448278899/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool1d(input, kernel_size, stride, padding, dilation, ceil_mode)


## Training

In [11]:
model.train(train_loader, epochs=config["epochs"])

Epoch 0


/home/jupyter/genomic_benchmarks/experiments/torch_cnn_experiments/utils.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.tensor(pad(x), dtype=torch.long)


Train metrics: 
 Accuracy: 69.5%, Avg loss: 0.629540 

Epoch 1
Train metrics: 
 Accuracy: 71.6%, Avg loss: 0.620643 

Epoch 2
Train metrics: 
 Accuracy: 76.6%, Avg loss: 0.610878 

Epoch 3
Train metrics: 
 Accuracy: 74.6%, Avg loss: 0.610260 

Epoch 4
Train metrics: 
 Accuracy: 76.3%, Avg loss: 0.605266 

Epoch 5
Train metrics: 
 Accuracy: 78.0%, Avg loss: 0.600325 

Epoch 6
Train metrics: 
 Accuracy: 78.6%, Avg loss: 0.605272 

Epoch 7
Train metrics: 
 Accuracy: 76.6%, Avg loss: 0.602380 

Epoch 8
Train metrics: 
 Accuracy: 78.5%, Avg loss: 0.597826 

Epoch 9
Train metrics: 
 Accuracy: 79.5%, Avg loss: 0.604997 

Epoch 10
Train metrics: 
 Accuracy: 79.2%, Avg loss: 0.595584 

Epoch 11
Train metrics: 
 Accuracy: 80.0%, Avg loss: 0.598875 

Epoch 12
Train metrics: 
 Accuracy: 79.7%, Avg loss: 0.597012 

Epoch 13
Train metrics: 
 Accuracy: 78.3%, Avg loss: 0.595518 

Epoch 14
Train metrics: 
 Accuracy: 80.9%, Avg loss: 0.592778 



## Testing

In [12]:
test_loader = DataLoader(test_dset, batch_size=config["batch_size"], shuffle=True, collate_fn=collate)

model.test(test_loader)

Unzipping...Done.
test_loss  579.0788348913193
num_batches 968
correct 24713
size 30970
Test Error: 
 Accuracy: 79.8%, Avg loss: 0.598222 

